In [1]:
import pandas as pd
import bs4 as bs 
import urllib.request
import re 

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/home/jyao01/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#Download the website's html
source = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_United_States_senators_in_the_117th_Congress").read()
soup = bs.BeautifulSoup(source)

In [4]:
urls_css = ".fn a"
urls = soup.select(urls_css)

In [5]:
names = [url.text for url in urls]
sites = [url["href"] for url in urls]
page_urls = ["https://en.wikipedia.org/" + site for site in sites]

In [6]:
len(page_urls)

102

In [7]:
s =[]

for index, i in enumerate(page_urls[:]):
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  img_css = ".infobox-image a img"  
  img = soup.select(img_css)
  s.extend(list(set([i['src'] for i in img])))

In [8]:
u = ["https:" + item for item in s if (('logo' in item)==False)&(('250px' in item)==False)]

In [9]:
df = pd.DataFrame(columns=['face_name', 'face_url'])

In [10]:
df['face_name']=names
df['face_url']=u

In [11]:
df.head()

,face_name,face_url
0,Patrick Leahy,https://upload.wikimedia.org/wikipedia/commons...
1,Chuck Grassley,https://upload.wikimedia.org/wikipedia/commons...
2,Mitch McConnell,https://upload.wikimedia.org/wikipedia/commons...
3,Richard Shelby,https://upload.wikimedia.org/wikipedia/commons...
4,Dianne Feinstein,https://upload.wikimedia.org/wikipedia/commons...


In [12]:
df.loc[df['face_name'].str.contains('Casey'), 'face_url'][25]

'https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Bob_Casey_Jr._official_photo.jpg/220px-Bob_Casey_Jr._official_photo.jpg'

### Get masterfile wmpids

In [13]:
person = pd.read_csv("../datasets/people/person_2022.csv")

In [14]:
s_sen = person.loc[person['currsen_2022']==1]

In [15]:
s_sen = s_sen[['full_name', 'wmpid']]
s_sen.head()

,full_name,wmpid
132,Alex Padilla,WMPID1456
201,Amy Klobuchar,WMPID1512
300,Angus King,WMPID1461
454,Ben Ray Lujan,WMPID1412
455,Ben Sasse,WMPID1487


In [16]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [17]:
match = fuzzy_merge(s_sen, df, 'full_name', 'face_name', threshold=70)

In [18]:
match['matches'].isna().sum()

0

In [19]:
# Get false positives in mutliple matches
match['face_name'] = match['matches'].str.split(pat = ',', expand = True)[0]

In [20]:
match.head()

,full_name,wmpid,matches,face_name
132,Alex Padilla,WMPID1456,Alex Padilla,Alex Padilla
201,Amy Klobuchar,WMPID1512,Amy Klobuchar,Amy Klobuchar
300,Angus King,WMPID1461,Angus King,Angus King
454,Ben Ray Lujan,WMPID1412,Ben Ray Luján,Ben Ray Luján
455,Ben Sasse,WMPID1487,Ben Sasse,Ben Sasse


In [21]:
match = match.merge(df, how='left', on='face_name')

In [22]:
match.head()

,full_name,wmpid,matches,face_name,face_url
0,Alex Padilla,WMPID1456,Alex Padilla,Alex Padilla,https://upload.wikimedia.org/wikipedia/commons...
1,Amy Klobuchar,WMPID1512,Amy Klobuchar,Amy Klobuchar,https://upload.wikimedia.org/wikipedia/commons...
2,Angus King,WMPID1461,Angus King,Angus King,https://upload.wikimedia.org/wikipedia/commons...
3,Ben Ray Lujan,WMPID1412,Ben Ray Luján,Ben Ray Luján,https://upload.wikimedia.org/wikipedia/commons...
4,Ben Sasse,WMPID1487,Ben Sasse,Ben Sasse,https://upload.wikimedia.org/wikipedia/commons...


In [23]:
# split error for face name Bob Casey, Jr.
match.loc[match['face_url'].isna()]

,full_name,wmpid,matches,face_name,face_url
77,Robert Casey,WMPID1520,"Bob Casey, Jr.",Bob Casey,NaN


In [24]:
match.loc[match['full_name']=='Robert Casey', 'face_url']=df.loc[df['face_name'].str.contains('Casey'), 'face_url'][25]

In [25]:
ss = match[['wmpid', 'face_url']]

In [26]:
ss['face_category'] = 'SitSen'

/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
ss.head()

,wmpid,face_url,face_category
0,WMPID1456,https://upload.wikimedia.org/wikipedia/commons...,SitSen
1,WMPID1512,https://upload.wikimedia.org/wikipedia/commons...,SitSen
2,WMPID1461,https://upload.wikimedia.org/wikipedia/commons...,SitSen
3,WMPID1412,https://upload.wikimedia.org/wikipedia/commons...,SitSen
4,WMPID1487,https://upload.wikimedia.org/wikipedia/commons...,SitSen


In [28]:
ss.shape

(100, 3)

In [29]:
ss.to_csv('./data/face_url_2022_sitting_senate.csv', index=False)